In [1]:
#Importa Bibliotecas

from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

#Mostrar todas as colunas
pd.set_option('display.max_columns', None)

#Mostrar os números com duas casa decimais sem notação cientifica
pd.set_option('display.float_format', '{:.2f}'.format)



In [2]:
#Chama página com todos os FIIS e armazena em uma lista

url_listafiis = "https://fiis.com.br/lista-de-fundos-imobiliarios/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
}

response = requests.get(url_listafiis, headers=headers)

if response.status_code == 200:
    
    fiis = BeautifulSoup(response.text, 'html.parser')

else:
    print(f"Erro ao acessar a página: {response.status_code}")

In [3]:
# Traz todos os Fiis e armazena em uma lista
recorta_nome_fiis = fiis.find_all("div", class_="tickerBox__title", attrs={"data-element": "ticker-box-title"})

nome_fiis = []
for fii in recorta_nome_fiis:
    nome_fiis.append(fii.text)

In [4]:
#Gera um link para cada FII de onde serão extraídas as informações

url_fiis = "https://fiis.com.br/"
url_fiis_completa = []

for i in nome_fiis:

    url_fiis_temp = url_fiis + i
    url_fiis_completa.append(url_fiis_temp)

In [5]:
#Traz informação de cada FII e armazena em suas respectivas listas

fii, indicadores, Dados, tabeladiv, detalhes_cota, indicadores_caixa, fiis, segundatentativa, erros = [], [], [], [], [], [], [], [], []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
}

for j in url_fiis_completa:

    retorno = requests.get(j, headers=headers, timeout=(5, 10))
    
    if retorno.status_code == 200:
    
        fii = BeautifulSoup(retorno.text, 'html.parser')
        indicadores = fii.find_all("div", class_="headerTicker__content__info")
        Dados = fii.find_all("div", class_="moreInfo wrapper")
        tabeladiv = fii.find_all("div", class_="yieldChart__table__bloco yieldChart__table__bloco--rendimento")
        detalhes_cota = fii.find("div", class_="quotations-details")
        indicadores_caixa = fii.find("div", class_="wrapper indicators")
        
        try:
        
            fiis.append({
            
            'Nome_FII': fii.find("h1").string,
            'Titulo': fii.title.string,
            'Administradora': fii.find("p", class_="informations__adm__name").text,
            'Dividendo_Yeld_ano': indicadores[0].find(string="Dividend Yield").find_previous("b").text.strip(),
            'Ultimo_Dividendo': indicadores[0].find(string="Último Rendimento").find_previous("b").text.strip(),
            'CNPJ': Dados[0].find("span", string="CNPJ").find_next("b").text.strip(),
            'Nome_Pregao': Dados[0].find("span", string="Nome no Pregão").find_next("b").text.strip(),
            'Qtd_Cotas': Dados[0].find("span", string="Número de Cotas").find_next("b").text.strip(),
            'Patrimonio': Dados[0].find("span", string="Patrimônio").find_next("b").text.strip(),
            'Segmento': Dados[0].find("span", string="Segmento").find_next("b").text.strip(),
            'Publico_alvo': Dados[0].find("span", string="Publico Alvo").find_next("b").text.strip(),
            'Cotacao_Atual': detalhes_cota.find("span", class_="value").text.strip(),
            'Valor_Caixa': indicadores_caixa.find(string="Valor em caixa").find_previous("b").text.strip(),
            'Liquidez_Media_Dia': indicadores_caixa.find(string="Liquidez média diária").find_previous("b").text.strip(),
            'Vlr_Patr_Cota': indicadores_caixa.find(string="Val. Patrimonial p/Cota").find_previous("b").text.strip(),
            'Qdt_Cotistas': indicadores_caixa.find(string="N° de Cotistas").find_previous("b").text.strip(),
            'Part_IFIX': indicadores_caixa.find(string="Participações no IFIX").find_previous("b").text.strip(),
            'Data_Base': tabeladiv[0].find_all("div", class_="table__linha")[1].text.strip(),
            'Data_PGg_Div': tabeladiv[0].find_all("div", class_="table__linha")[2].text.strip(),
            'Cotacao_Base': tabeladiv[0].find_all("div", class_="table__linha")[3].text.strip(),
            'Dividend_Yeld': tabeladiv[0].find_all("div", class_="table__linha")[4].text.strip()
            
            })            
        
        except:
        
            fiis.append({
            
            'Nome_FII': fii.find("h1").string,
            'Titulo': fii.title.string,
            'Administradora': fii.find("p", class_="informations__adm__name").text,
            'Dividendo_Yeld_ano': indicadores[0].find(string="Dividend Yield").find_previous("b").text.strip(),
            'Ultimo_Dividendo': indicadores[0].find(string="Último Rendimento").find_previous("b").text.strip(),
            'CNPJ': Dados[0].find("span", string="CNPJ").find_next("b").text.strip(),
            'Nome_Pregao': Dados[0].find("span", string="Nome no Pregão").find_next("b").text.strip(),
            'Qtd_Cotas': Dados[0].find("span", string="Número de Cotas").find_next("b").text.strip(),
            'Patrimonio': Dados[0].find("span", string="Patrimônio").find_next("b").text.strip(),
            'Segmento': Dados[0].find("span", string="Segmento").find_next("b").text.strip(),
            'Publico_alvo': Dados[0].find("span", string="Publico Alvo").find_next("b").text.strip(),
            'Cotacao_Atual': detalhes_cota.find("span", class_="value").text.strip(),
            'Valor_Caixa': indicadores_caixa.find(string="Valor em caixa").find_previous("b").text.strip(),
            'Liquidez_Media_Dia': indicadores_caixa.find(string="Liquidez média diária").find_previous("b").text.strip(),
            'Vlr_Patr_Cota': indicadores_caixa.find(string="Val. Patrimonial p/Cota").find_previous("b").text.strip(),
            'Qdt_Cotistas': indicadores_caixa.find(string="N° de Cotistas").find_previous("b").text.strip(),
            'Part_IFIX': indicadores_caixa.find(string="Participações no IFIX").find_previous("b").text.strip(),
            'Data_Base': "",
            'Data_PGg_Div': "",
            'Cotacao_Base': "",
            'Dividend_Yeld': ""
            
            })
        
    
    elif retorno.status_code == 404:
    
        segundatentativa = j.replace("11" , "")
        
        retorno = requests.get(segundatentativa, headers=headers, timeout=(5, 10))
        
        if retorno.status_code == 200:
        
            fii = BeautifulSoup(retorno.text, 'html.parser')
            indicadores = fii.find_all("div", class_="headerTicker__content__info")
            Dados = fii.find_all("div", class_="moreInfo wrapper")
            tabeladiv = fii.find_all("div", class_="yieldChart__table__bloco yieldChart__table__bloco--rendimento")
            detalhes_cota = fii.find("div", class_="quotations-details")
            indicadores_caixa = fii.find("div", class_="wrapper indicators")
            
            try:
            
                fiis.append({
                
                'Nome_FII': fii.find("h1").string,
                'Titulo': fii.title.string,
                'Administradora': fii.find("p", class_="informations__adm__name").text,
                'Dividendo_Yeld_ano': indicadores[0].find(string="Dividend Yield").find_previous("b").text.strip(),
                'Ultimo_Dividendo': indicadores[0].find(string="Último Rendimento").find_previous("b").text.strip(),
                'CNPJ': Dados[0].find("span", string="CNPJ").find_next("b").text.strip(),
                'Nome_Pregao': Dados[0].find("span", string="Nome no Pregão").find_next("b").text.strip(),
                'Qtd_Cotas': Dados[0].find("span", string="Número de Cotas").find_next("b").text.strip(),
                'Patrimonio': Dados[0].find("span", string="Patrimônio").find_next("b").text.strip(),
                'Segmento': Dados[0].find("span", string="Segmento").find_next("b").text.strip(),
                'Publico_alvo': Dados[0].find("span", string="Publico Alvo").find_next("b").text.strip(),
                'Cotacao_Atual': detalhes_cota.find("span", class_="value").text.strip(),
                'Valor_Caixa': indicadores_caixa.find(string="Valor em caixa").find_previous("b").text.strip(),
                'Liquidez_Media_Dia': indicadores_caixa.find(string="Liquidez média diária").find_previous("b").text.strip(),
                'Vlr_Patr_Cota': indicadores_caixa.find(string="Val. Patrimonial p/Cota").find_previous("b").text.strip(),
                'Qdt_Cotistas': indicadores_caixa.find(string="N° de Cotistas").find_previous("b").text.strip(),
                'Part_IFIX': indicadores_caixa.find(string="Participações no IFIX").find_previous("b").text.strip(),
                'Data_Base': tabeladiv[0].find_all("div", class_="table__linha")[1].text.strip(),
                'Data_PGg_Div': tabeladiv[0].find_all("div", class_="table__linha")[2].text.strip(),
                'Cotacao_Base': tabeladiv[0].find_all("div", class_="table__linha")[3].text.strip(),
                'Dividend_Yeld': tabeladiv[0].find_all("div", class_="table__linha")[4].text.strip()
                
                })            
            
            except:
            
                fiis.append({
                
                'Nome_FII': fii.find("h1").string,
                'Titulo': fii.title.string,
                'Administradora': fii.find("p", class_="informations__adm__name").text,
                'Dividendo_Yeld_ano': indicadores[0].find(string="Dividend Yield").find_previous("b").text.strip(),
                'Ultimo_Dividendo': indicadores[0].find(string="Último Rendimento").find_previous("b").text.strip(),
                'CNPJ': Dados[0].find("span", string="CNPJ").find_next("b").text.strip(),
                'Nome_Pregao': Dados[0].find("span", string="Nome no Pregão").find_next("b").text.strip(),
                'Qtd_Cotas': Dados[0].find("span", string="Número de Cotas").find_next("b").text.strip(),
                'Patrimonio': Dados[0].find("span", string="Patrimônio").find_next("b").text.strip(),
                'Segmento': Dados[0].find("span", string="Segmento").find_next("b").text.strip(),
                'Publico_alvo': Dados[0].find("span", string="Publico Alvo").find_next("b").text.strip(),
                'Cotacao_Atual': detalhes_cota.find("span", class_="value").text.strip(),
                'Valor_Caixa': indicadores_caixa.find(string="Valor em caixa").find_previous("b").text.strip(),
                'Liquidez_Media_Dia': indicadores_caixa.find(string="Liquidez média diária").find_previous("b").text.strip(),
                'Vlr_Patr_Cota': indicadores_caixa.find(string="Val. Patrimonial p/Cota").find_previous("b").text.strip(),
                'Qdt_Cotistas': indicadores_caixa.find(string="N° de Cotistas").find_previous("b").text.strip(),
                'Part_IFIX': indicadores_caixa.find(string="Participações no IFIX").find_previous("b").text.strip(),
                'Data_Base': "",
                'Data_PGg_Div': "",
                'Cotacao_Base': "",
                'Dividend_Yeld': ""
                
                })
        
        
        else:
            erro = (f"{j} : {retorno}")
            erros.append(erro)
    
    else:
        
        erro = (f"{j} : {retorno}")
        erros.append(erro)
    
    time.sleep(1)

In [6]:
#criando DataFrame

df_fiss = pd.DataFrame(fiis) 

#Trata Colunas
df_fiss['Dividendo_Yeld_ano'] = df_fiss['Dividendo_Yeld_ano'].astype(str).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float) 
df_fiss['Ultimo_Dividendo'] = df_fiss['Ultimo_Dividendo'].astype(str).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float) 
df_fiss['Patrimonio'] = df_fiss['Patrimonio'].astype(str).replace(r'R\$ ', '', regex=True).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)
df_fiss['Cotacao_Atual'] = df_fiss['Cotacao_Atual'].astype(str).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)
df_fiss['Vlr_Patr_Cota'] = df_fiss['Vlr_Patr_Cota'].astype(str).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)
df_fiss['Qdt_Cotistas'] = df_fiss['Qdt_Cotistas'].astype(str).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(int)
df_fiss['Part_IFIX'] = df_fiss['Part_IFIX'].astype(str).replace(r'N\/A', '', regex=True).replace(r'\%', '', regex=True).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)
df_fiss["Data_Base"] = pd.to_datetime(df_fiss["Data_Base"], format='%d.%m.%Y')
df_fiss["Data_PGg_Div"] = pd.to_datetime(df_fiss["Data_PGg_Div"], format='%d.%m.%Y')
df_fiss['Cotacao_Base'] = df_fiss['Cotacao_Base'].astype(str).replace(r'R\$ ', '', regex=True).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)
df_fiss['Dividend_Yeld'] = df_fiss['Dividend_Yeld'].astype(str).replace(r'N\/A', '', regex=True).replace(r'\%', '', regex=True).replace(r'\.', '', regex=True).replace(',', '.', regex=True).replace('', 'NaN').astype(float)

#Cria colunas Calculadas
df_fiss['Valor_Caixa_Calculado'] = df_fiss['Valor_Caixa'].apply(lambda x: 
    None if x == '-' else 
    (float(x.replace('M', '').replace('.', '').replace(',', '.')) * 1000000 if x[-1] == 'M' else 
     (float(x.replace('K', '').replace('.', '').replace(',', '.')) * 1000 if x[-1] == 'K' else 
      (float(x.replace('B', '').replace('.', '').replace(',', '.')) * 1000000000 if x[-1] == 'B' else 
       float(x.replace('.', '').replace(',', '.'))))))
df_fiss['Liquidez_Media_Dia_Calculado'] = df_fiss['Liquidez_Media_Dia'].apply(lambda x: 
    None if x == '-' else 
    (float(x.replace('M', '').replace('.', '').replace(',', '.')) * 1000000 if x[-1] == 'M' else 
     (float(x.replace('K', '').replace('.', '').replace(',', '.')) * 1000 if x[-1] == 'K' else 
      (float(x.replace('B', '').replace('.', '').replace(',', '.')) * 1000000000 if x[-1] == 'B' else 
       float(x.replace('.', '').replace(',', '.'))))))




In [7]:
df_fiss

,Nome_FII,Titulo,Administradora,Dividendo_Yeld_ano,Ultimo_Dividendo,CNPJ,Nome_Pregao,Qtd_Cotas,Patrimonio,Segmento,Publico_alvo,Cotacao_Atual,Valor_Caixa,Liquidez_Media_Dia,Vlr_Patr_Cota,Qdt_Cotistas,Part_IFIX,Data_Base,Data_PGg_Div,Cotacao_Base,Dividend_Yeld,Valor_Caixa_Calculado,Liquidez_Media_Dia_Calculado
0,AAGR11,AAGR11 - FIAGRO AAGR - Fiis,,0.00,0.00,44.585.001/0001-10,FIAGRO AAGR,5.000.000,0.00,,,98.98,-,"8,0 K",0.00,0,NaN,2024-10-08,2024-10-15,96.92,1.11,NaN,8000.00
1,AAZQ11,AAZQ11 - FIAGRO AAZQ - Fiis,XP INVESTIMENTOS CCTVM S.A.,15.66,0.11,44.625.826/0001-11,FIAGRO AAZQ,24.037.284,230857797.00,Fiagro,Investidores em Geral,6.65,"26,4 M","780,4 K",9.60,27598,NaN,2024-10-31,2024-11-14,6.73,1.41,26400000.00,780400.00
2,ABCP11,ABCP11 - Grand Plaza Shopping - Fiis,RIO BRAVO INVESTIMENTOS - DISTRIBUIDORA DE TIT...,9.61,0.47,01.201.140/0001-90,Grand Plaza Shopping,4.709.082,477377336.00,Shoppings,Investidores em Geral,70.11,"7,6 M","60,9 K",101.37,16751,NaN,2024-10-31,2024-11-07,77.00,0.91,7600000.00,60900.00
3,AFCR11,AFCR11 - Fiis,,0.00,0.00,,,0,0.00,Indefinido,,0.00,-,-,0.00,0,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,AFHI11,AFHI11 - FII AFHI CRI - Fiis,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,13.80,0.95,36.642.293/0001-58,FII AFHI CRI,3.996.951,385438733.00,Papéis,Investidores em Geral,97.20,"30,9 M","1,5 M",96.43,32625,0.32,2024-10-14,2024-10-21,94.75,0.98,30900000.00,1500000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,YUFI11,YUFI11 - FII YUCA - Fiis,BRL TRUST DISTRIBUIDORA DE TITULOS E VALORES M...,3.49,0.39,36.445.551/0001-06,FII YUCA,492.650,49493776.00,Fundo de Desenvolvimento,Investidor Profissional,90.00,"949,5 K","3,1 K",100.46,63,NaN,2024-10-31,2024-11-14,88.97,0.11,949500.00,3100.00
659,ZAGH11,ZAGH11 - Fiis,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,3.46,0.07,45.188.066/0001-95,,4.645.960,44483015.00,,Investidor Qualificado,13.00,"5,9 M","1,2 K",9.57,261,NaN,2024-10-11,2024-10-18,11.98,0.58,5900000.00,1200.00
660,ZAVC11,ZAVC11 - FII ZAVIT C - Fiis,VORTX DISTRIBUIDORA DE TITULOS E VALORES MOBIL...,6.70,0.13,52.101.897/0001-43,FII ZAVIT C,3.097.348,31271344.00,,Investidor Profissional,10.30,"4,3 M","1,5 K",10.10,130,NaN,2024-10-31,2024-11-13,9.00,1.44,4300000.00,1500.00
661,ZAVI11,ZAVI11 - FII-ZAVIT-R - Fiis,VORTX DISTRIBUIDORA DE TITULOS E VALORES MOBIL...,13.84,1.15,40.575.940/0001-23,FII-ZAVIT-R,1.173.322,154456975.00,Indefinido,Investidores em Geral,104.00,"9,3 M","112,3 K",131.64,4112,0.09,2024-10-31,2024-11-13,104.98,1.10,9300000.00,112300.00
